## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

1) Откроем файл с данными и изучим общую информацию. 

In [14]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
#print(data.head(50))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

данных в столбцах days_employed и total_income меньше, т.е. скорее всего есть пропуски. смущает тип float в этих же столбцах. также надо обоатить внимание на данные в столбце days_employed

2) Предобработка данных

### Обработка пропусков

In [15]:
#print(data['total_income'].isnull().sum())

mean_total_income = data.groupby('income_type')['total_income'].mean()

data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'сотрудник'), 'total_income'] = mean_total_income[6]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'компаньон'), 'total_income'] = mean_total_income[3]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'пенсионер'), 'total_income'] = mean_total_income[4]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'госслужащий'), 'total_income'] = mean_total_income[2]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'предприниматель'), 'total_income'] = mean_total_income[5]


#data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))



income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64


### Вывод

пропуски есть в days_employed(общий трудовой стаж) и total_income (ежемесячный доход). для задания нас интересует total_income, поэтому для заполнения пропусков найдем средний заработок по роду деятельности (income_type) и именно средним значением заполним пропуски

### Замена типа данных

In [16]:


#data['total_income'] = data['total_income'].astype('int')
data['total_income'] = pd.to_numeric(data['total_income'], errors = 'coerce')
data['total_income'] = data['total_income'].astype('int')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

сначала я перевел все строковые значения в числа, а затем float перевел в int (нас интересует столбец total_income)

### Обработка дубликатов

In [17]:

data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

#print(data.duplicated().sum())
#data["is_duplicate"] = data.duplicated(keep=False)
#data_filter_dublicates = data[ (data['is_duplicate'] == True)]
#print(data_filter_dublicates.sort_values(by='total_income').head(25))
data = data.drop_duplicates().reset_index(drop = True)

#Еще один способо как быстро переименовать все колонки
#data.columns = [col.lower().replace(' ', '_') for col in data.columns]


### Вывод

приведя к единому регистру 'education' мы упростим себе задачу в будушем. 

### Лемматизация

In [18]:
from pymystem3 import Mystem
m = Mystem()

purposes = ', '.join(data['purpose'].unique())
lemm_purposes = m.lemmatize(purposes)
data['lem_purpose'] = data['purpose'].apply(m.lemmatize)
from collections import Counter
#print(Counter(lemm_purposes))
lemm_purposes = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба']
def find_purpose(data):
    
    for purpose in lemm_purposes:
           if purpose in data: 
            return purpose
data['new_purpose'] = data['lem_purpose'].apply(find_purpose)
print(data['new_purpose'].value_counts())



недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: new_purpose, dtype: int64


### Вывод

нам нужно понять для каких целей брали кредит. естественно все заемщики по разному написали цели, но если мы посморим уникальные значения столбца 'purpose', то уидим, что есть много похожих целей, нам надо их выделить. для этого лемматизируем данные столбца 'purpose'. чтобы lemmatize вопринимала данные столбца, нужно привести его в строчное значение с помощью метода .join(). затем применить лемматизацию и посчитать количество уникальных слов. после этого составим из нужных слов (целей) словарь и к нему применим функцию, которая отсортирует лемматизированныне данные столбца purpose по словам из нашего словаря

### Категоризация данных

In [ ]:
def parenthood(children):
    if children == 0:
        return '0'
    if children == 1:
        return '1'
    if children == -1:
        return '1'
    if children == 2:
        return '2'
    if children == 20:
        return '2'
    if children == 3:
        return '3'
    if children == 4:
        return '4'
    if children == 5:
        return '5'
    return 'ошибка'

data['parenthood'] = data['children'].apply(parenthood)

#print(data['total_income'].max()) - 2265604
#print(data['total_income'].min()) - 20667
#print(data['total_income'].median()) - 151887.0
#print(len(data)) - 21454

#print(data['total_income'].value_counts().head(50)): 1071 клиентов имеют заработок 161380; 501 - 202417; 386 - 137127; 145 - 170898
#поэтому предположим, что планка срднего дохода 200000, а меньше 100000 это низкий доход

def income_type (total_income):
    if total_income <= 100000:
        return 'низкий доход'
    if total_income <= 200000:
        return 'средний доход'
    return 'высокий доход'
data['income_level'] = data['total_income'].apply(income_type)

print(data['income_level'].value_counts())
inc_purp = data[['income_level', 'new_purpose']]




### Вывод

в столбце children имеются два страных показателя: -1 и 20. предположим, что это ошибка оператора и -1 это 1, а 20 это 2
добавим в data новый столбец parenthood с измененными данными

3) Попробуем ответить на некоторые вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [126]:
data_grouped = data.groupby('parenthood').agg({'debt':'mean'})
#print(data_grouped.sort_values(by = 'debt', ascending = False)) - 4           0.097561; 2           0.094925; 1           0.091658; 3           0.081818; 0           0.075438; 5           0.000000

four = 0.097561
two = 0.094925
one = 0.091658
three = 0.081818
zero = 0.075438
five = 0.000000
print('Четыре ребенка: {:.1%}'.format(four))
print('Два ребенка: {:.1%}'.format(two))
print('Один ребенок: {:.1%}'.format(one))
print('Три ребенка: {:.1%}'.format(three))
print('Нет детей: {:.1%}'.format(zero))
print('Пять детей: {:.1%}'.format(five))


Четыре ребенка: 9.8%
Два ребенка: 9.5%
Один ребенок: 9.2%
Три ребенка: 8.2%
Нет детей: 7.5%
Пять детей: 0.0%


### Вывод

получается, что семьи с детьми действительно чаще испытывают проблемы с погашением кредитов в срок

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [125]:
data_grouped_fam = data.groupby('family_status').agg({'debt' : 'mean'})
#print(data_grouped_fam) - в разводе              0.071130; вдовец / вдова         0.065693; гражданский брак       0.093471;женат / замужем        0.075452;не женат / не замужем  0.097509
div = 0.071130
wid = 0.065693
civ = 0.093471
mar = 0.075452
single = 0.097509

print('В разводе: {:.1%}'.format(div))
print('Вдова/вдовец: {:.1%}'.format(wid))
print('Гражданский брак: {:.1%}'.format(civ))
print('Женат/замужем: {:.1%}'.format(mar))
print('Не женат/не замужем: {:.1%}'.format(single))




        

В разводе: 7.1%
Вдова/вдовец: 6.6%
Гражданский брак: 9.3%
Женат/замужем: 7.5%
Не женат/не замужем: 9.8%


### Вывод

у людей, которые состоят/состояли (в разводе, вдовы) в браке гораздо меньше проблем с погашением кредита

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [124]:
data_grouped_inc = data.groupby('income_level').agg({'debt' : 'mean'})

#print(data_grouped_inc) - высокий доход  0.069684; низкий доход   0.079319; средний доход  0.087455
low = 0.079319
middle = 0.087455
high = 0.069684
print('Низкий доход: {:.1%}'.format(low))
print('Средний доход: {:.1%}'.format(middle))
print('Высокий доход: {:.1%}'.format(high))

Низкий доход: 7.9%
Средний доход: 8.7%
Высокий доход: 7.0%


### Вывод

Прямой зависимости нет, но можно предположить, что люди с низким доходом более обдуманно подходят к кредитам

- Как разные цели кредита влияют на его возврат в срок?

In [122]:
#pivot_table
data_grouped_purp = data.groupby('new_purpose').agg({'debt' : 'mean'})
#print(data_grouped_purp) - автомобиль    0.093590; жилье         0.069058;недвижимость  0.074634;образование   0.092200;свадьба       0.080034
car = 0.093590
home = 0.069058
est = 0.074634
edu = 0.092200
wedding = 0.080034
print('Авто: {:.1%}'.format(car))
print('Жилье: {:.1%}'.format(home))
print('Недвижимость: {:.1%}'.format(est))
print('Образование: {:.1%}'.format(edu))
print('Свадьба: {:.1%}'.format(wedding))


Авто: 9.4%
Жилье: 6.9%
Недвижимость: 7.5%
Образование: 9.2%
Свадьба: 8.0%


### Вывод

Те, кто брал кредит для покупки автомобиля испытывали больше всего трудностей с выплатами. Не все хорошо и с кредитами на образование. А вот кредиты на жилье и на недвижимость выплачивались более исправно. Получается, что более крупные кредиты - а кредиты на операции с недвижимостью и жильем как правило крупные - брались людьми, которые могут стабильно платить деньги, что логично

4) Общий вывод

Можно сказать, что семейное положение и наличие детй влияют на способность отдавать кредиты. Наличие детей, судя по цифрам, действительно влияет в худшую сторону на погашение кредитов, но брак наоборот - в лучшую.  